-------BFS----------

In [3]:
from collections import deque

# Possible moves: up, down, left, right
MOVES = [(-1, 0), (1, 0), (0, -1), (0, 1)]

# Check if the next cell is inside bounds, walkable, and not yet visited
def can_visit(board, r, c, seen):
    rows, cols = len(board), len(board[0])
    return 0 <= r < rows and 0 <= c < cols and board[r][c] == 0 and (r, c) not in seen

# Trace back the path using the parent dictionary
def build_path(parent_map, target):
    route = []
    while target in parent_map:
        route.append(target)
        target = parent_map[target]
    route.append(target)  # add start
    return route[::-1]

# BFS for shortest path in grid
def shortest_path_bfs(board, start_pos, end_pos):
    q = deque([start_pos])
    seen = {start_pos}
    parent_map = {}

    while q:
        cur = q.popleft()

        if cur == end_pos:
            path_taken = build_path(parent_map, end_pos)
            return path_taken, len(path_taken) - 1  # cost is edges count

        for dr, dc in MOVES:
            nr, nc = cur[0] + dr, cur[1] + dc
            next_cell = (nr, nc)

            if can_visit(board, nr, nc, seen):
                q.append(next_cell)
                seen.add(next_cell)
                parent_map[next_cell] = cur

    return None, -1  # No valid route found

# Example usage
if __name__ == "__main__":
    maze = [
        [0, 1, 0, 0, 0],
        [0, 1, 0, 1, 0],
        [0, 0, 0, 1, 0],
        [1, 1, 0, 1, 0],
        [0, 0, 0, 0, 0]
    ]

    start_cell = (0, 0)
    goal_cell = (4, 4)

    route, steps = shortest_path_bfs(maze, start_cell, goal_cell)

    if route:
        print("Shortest Path:", route)
        print("Steps Required:", steps)
    else:
        print("No possible path found.")


Shortest Path: [(0, 0), (1, 0), (2, 0), (2, 1), (2, 2), (3, 2), (4, 2), (4, 3), (4, 4)]
Steps Required: 8


------DFS--------

In [4]:
# Directions: up, down, left, right
MOVES = [(-1, 0), (1, 0), (0, -1), (0, 1)]

# Check if the cell is inside bounds, walkable, and not visited yet
def can_move(board, r, c, seen):
    max_r, max_c = len(board), len(board[0])
    return 0 <= r < max_r and 0 <= c < max_c and board[r][c] == 0 and (r, c) not in seen

# Rebuild the route from goal to start using parent mapping
def trace_route(parent_map, goal_node):
    route = []
    while goal_node in parent_map:
        route.append(goal_node)
        goal_node = parent_map[goal_node]
    route.append(goal_node)  # Add the starting cell
    return route[::-1]

# Depth-First Search implementation
def dfs_pathfinder(board, start_cell, end_cell):
    stack = [start_cell]
    seen = {start_cell}
    parent_map = {}

    while stack:
        cur_cell = stack.pop()

        if cur_cell == end_cell:
            full_path = trace_route(parent_map, end_cell)
            return full_path, len(full_path) - 1

        for dr, dc in MOVES:
            nr, nc = cur_cell[0] + dr, cur_cell[1] + dc
            next_cell = (nr, nc)

            if can_move(board, nr, nc, seen):
                stack.append(next_cell)
                seen.add(next_cell)
                parent_map[next_cell] = cur_cell

    return None, -1  # No path exists

# Example usage
if __name__ == "__main__":
    maze = [
        [0, 1, 0, 0, 0],
        [0, 1, 0, 1, 0],
        [0, 0, 0, 1, 0],
        [1, 1, 0, 1, 0],
        [0, 0, 0, 0, 0]
    ]

    start_point = (0, 0)
    goal_point = (4, 4)

    path_result, step_count = dfs_pathfinder(maze, start_point, goal_point)

    if path_result:
        print("DFS Path:", path_result)
        print("Steps Taken:", step_count)
    else:
        print("No path found.")


DFS Path: [(0, 0), (1, 0), (2, 0), (2, 1), (2, 2), (3, 2), (4, 2), (4, 3), (4, 4)]
Steps Taken: 8


------A*---------

In [5]:
import heapq

# Movements: up, down, left, right
MOVES = [(-1, 0), (1, 0), (0, -1), (0, 1)]

# Manhattan distance heuristic
def manhattan_dist(p1, p2):
    return abs(p1[0] - p2[0]) + abs(p1[1] - p2[1])

# Check if the position is within bounds and not blocked
def can_travel(map_grid, r, c):
    rows, cols = len(map_grid), len(map_grid[0])
    return 0 <= r < rows and 0 <= c < cols and map_grid[r][c] == 0

# Backtrack to build the path
def build_path(parent_map, end_node):
    route = []
    while end_node in parent_map:
        route.append(end_node)
        end_node = parent_map[end_node]
    route.append(end_node)
    return route[::-1]

# A* Pathfinding
def a_star_search(map_grid, start_node, target_node):
    open_queue = [(manhattan_dist(start_node, target_node), 0, start_node)]  # (f, g, node)
    heapq.heapify(open_queue)

    came_from = {}
    g_costs = {start_node: 0}
    explored = set()

    while open_queue:
        _, g_val, current_node = heapq.heappop(open_queue)

        if current_node == target_node:
            final_path = build_path(came_from, target_node)
            return final_path, g_val

        if current_node in explored:
            continue
        explored.add(current_node)

        for dr, dc in MOVES:
            nr, nc = current_node[0] + dr, current_node[1] + dc
            neighbor = (nr, nc)

            if can_travel(map_grid, nr, nc):
                new_g = g_val + 1
                if neighbor not in g_costs or new_g < g_costs[neighbor]:
                    g_costs[neighbor] = new_g
                    f_val = new_g + manhattan_dist(neighbor, target_node)
                    came_from[neighbor] = current_node
                    heapq.heappush(open_queue, (f_val, new_g, neighbor))

    return None, -1  # No valid path

# Example usage
if __name__ == "__main__":
    maze_map = [
        [0, 1, 0, 0, 0],
        [0, 1, 0, 1, 0],
        [0, 0, 0, 1, 0],
        [1, 1, 0, 1, 0],
        [0, 0, 0, 0, 0]
    ]

    start_pos = (0, 0)
    goal_pos = (4, 4)

    path_found, path_cost = a_star_search(maze_map, start_pos, goal_pos)

    if path_found:
        print("A* Path:", path_found)
        print("Total Cost:", path_cost)
    else:
        print("No path found.")


A* Path: [(0, 0), (1, 0), (2, 0), (2, 1), (2, 2), (3, 2), (4, 2), (4, 3), (4, 4)]
Total Cost: 8
